# 1 Preliminary Data Cleaning, Preprocessing and EDA

- Author: Jason Truong
- Last Modified: September 9, 2022
- Email: Jasontruong19@gmail.com

# Table of Contents

1. [Objective](#1Objective)  
2. [Preliminary Data Cleaning](#2Preliminary)  
    2.1. [Preprocess Duplicated Rows]()  
    2.2. [Check NaNs](#)  
    2.3. [Preprocess: Duplicated Columns](#)  
    2.1. [Preprocessing: 'Overall'](#2_1Overall)  
    2.2. [Preprocessing: 'reviewScore'](#2_2Review)  
    2.3. [Preprocessing: 'Vote'](#2_3Vote)  
    2.4. [Drop duplicates and NaNs](#2_4Drop)  
3. [Exploratory Data Analysis](#3EDA)  
4. [Simple Modeling](#4Simple_Model)  

# 1. Objective<a class ='anchor' id='1Objective'></a>

Reviews provide users with feedback about whether a purchased product is good or not. Reviews play a big role in influencing peoples' decision to purchase a product. In this notebook, the Amazon review data will be loaded in, preproccessed and cleaned for further analysis. The cleaned review data will also be used for a preliminary exploratory data analysis to expose any trends in the data.

# 2. Preliminary Data Cleaning <a class ='anchor' id='2Preliminary'></a>

In [ ]:
# Import necessary packages.
import json
from datetime import datetime
import pandas as pd
import numpy as np

# Plotting package
import matplotlib.pyplot as plt
import seaborn as sns

Since the dataset contains 8.9 million rows of data and is 5.2 gbs, only a small subset of the data will be loaded in.

Load in the dataset

In [ ]:
review_data = []

# Read in the first 2,000,000 rows from the dataset
with open('Movies_and_TV.json','r') as metafile:
    for x in range(2000000):
        review_data.append(json.loads(next(metafile)))

Create the dataframe and explore the data

In [ ]:
review_df = pd.DataFrame(review_data)

In [ ]:
review_df.info(show_counts= True)

There are 12 columns of data with 9 object columns that will been to be convereted to numeric

## Drop any duplicated rows in the dataframe

In [ ]:
review_df.drop_duplicates(inplace = True, ignore_index = True)

In [ ]:
review_df.info()

1365 entries has been dropped.

### Check NaNs

In [ ]:
review_df.isna().sum()

There are 1497 NaNs in the reviewText column and 394 NaNs in the summary column. This represents a very very small proportion of the data since there are ~ 2 million rows so these rows will just be dropped. Also, the reviewText is the main feature for the machine learning models thus, the rows that have NaNs for ReviewText will essentially be useless for the NLP analysis use case.

In [ ]:
review_df.dropna(inplace= True)

# Check results
review_df.isna().sum()

In [ ]:
review_df.info()

## Preprocess Duplicated Columns

In [ ]:
review_df.head()

Looking at the columns, `reviewTime` and `unixReviewTime` may display the same information, a check will be completed to verify if they display the same information. The `reviewerName` does not provide any useful information since there is a unique `reviewerID` for each reviewer. Two reviewers can have the same name but may not be the same person. Thus `reviewerName` will be dropped. The `image` column can be dropped because this project will not deal with image data. 

**Before the `reviewerName` is dropped, it will be compared with the reviewerID column to ensure they contain the same information**

In [ ]:
review_df['reviewerID'].value_counts()

In [ ]:
review_df['reviewerName'].value_counts()

The `reviewerID` and `reviewerName` column show differrent lengths. The `reviewerName` column contains less information than the `reviewerID` column because users can have the same names but reviewerID is unique. Thus reviewerID should be used.

**Verify that the `unixReviewTime` is the same as the `reviewTime` column. First the `reviewTime` will be converted to a datetime type.**

In [ ]:
# Convert reviewTime column to datetime type
review_df['reviewTime'] = pd.to_datetime(review_df['reviewTime'])

# Check results
review_df['reviewTime'].dtype

Next, convert the unixReviewTime to a datetime format.

In [ ]:
# Convert unixReviewTime column to datetime type
review_df['unixReviewTime'] = pd.to_datetime(review_df['unixReviewTime'], unit = 's')

# Check results
review_df['unixReviewTime']

Check if the column values are the same.

In [ ]:
(review_df['reviewTime'] == review_df['unixReviewTime']).sum()

All rows of dates are identical so one of them can be dropped. ReviewTime will be dropped.

**The unnecessary columns can be dropped from the dataframe.**

In [ ]:
# Drop the unnecessary columns
review_df.drop(columns = ['reviewTime','reviewerName','image'],inplace = True)

# Check results
review_df

In [ ]:
# Check results
review_df.info()

### Check column `overall`

In [ ]:
review_df['overall'].value_counts()

It looks like the values for the overall column are contained between 1 and 5 which makes sense since the reviews are out of 5. This column represents the review score so it will be renamed for clarity.

In [ ]:
# Rename column
review_df.rename(columns={'overall':'reviewScore'}, inplace = True)

# Check results
review_df

### Check the column `Vote`

For the `vote` column, since the NaN values are essentially no votes which can be represented with 0, the NaN values will be replaced with a zero. There are also commas within the vote that causes problems when converted to an int so they will be removed.

In [ ]:
# Remove commas
review_df['vote'] = review_df['vote'].str.replace(r"\,","",regex = True)

# Fill NaNs with 0
review_df['vote'] = review_df['vote'].fillna(0)

# Change datatype to int
review_df['vote'] = review_df['vote'].astype('intz')

# Check results
review_df.head()

### Split the reviewTime into date, month and year

The date column will be split up to determine if the data shows any trends in the days, months and years that the reviews were made.

In [ ]:
# Extract the day information
review_df['reviewDay'] = review_df['unixReviewTime'].dt.day

# Extract the month information
review_df['reviewMonth'] = review_df['unixReviewTime'].dt.month

# Extract the year information
review_df['reviewYear'] = review_df['unixReviewTime'].dt.year

# Check results
review_df.head()

Drop the unixReviewTime column.

In [ ]:
review_df.drop(columns ='unixReviewTime', inplace = True)

# Check results
review_df

### Check the column `style`

The NaN values in the column `style` will be filled with 'other' since the values is unknown.
The column style will be converted to strings so that a duplicate check can be performed for the dataset. (Converting to str, then removing the key for the dictionary is more computationally effective for getting the value in the key-value pair than looping through every individual key and accessing the value. *Note: the latter was tried and took 10 mins for 1 million key-value pairs. The former took <5s.)

In [ ]:
review_df['style'] = review_df['style'].fillna('Unknown')

In [ ]:
# Change values in the style column from dict to str
review_df['style'] = review_df['style'].astype('str')

In [ ]:
# Remove the keys in the key-value pair
review_df['style'] = review_df['style'].str.replace(r"{'Format:': ' ","",regex = True)
review_df['style'] = review_df['style'].str.replace(r"'}","",regex = True)

# Check results
review_df['style']

The style column now contains the relavent data.

Check which styles should be kept by checking the frequency of the different style entries

In [ ]:
review_df['style'].value_counts()

It looks like the top 5 styles represent the majority of the data.

In [ ]:
# Check the percentage the top 5 styles represent
review_df['style'].value_counts().head(5).sum()/review_df.shape[0]

The top 5 styles represent 99.6% of the data so group everything else to "other" and transform this column to one hot encoded variables.

In [ ]:
styles_keep = review_df['style'].value_counts().index[:5]

Replace all the other styles with "Other"

In [ ]:
# If the style is in the top 5 styles, keep it, else change it to Other
review_df['style'] = np.where(review_df['style'].isin(styles_keep), 
                              review_df['style'], 
                              "Other")

Check the style column

In [ ]:
review_df['style'].value_counts()

The transformation was successful and now there are only 6 style options. The next step is to dummy encode this column and drop the `unknown` column because that column contained NaN values to start off with. One column has to be dropped to ensure that there is no multicollinearity when we use the dummy style columns in the models in the future.

In [ ]:
# Use get_dummies function, drop the 'unknown' column and 
# add the prefix 'style_' to known which column these dummy variables came from

style_dummies = pd.get_dummies(review_df['style']).drop(columns = 'Unknown').add_prefix('style_')
style_dummies

The style column was successfully encoded into dummy variables so now combine it with the original dataframe.

In [ ]:
review_df = pd.concat([review_df,style_dummies], axis = 1)
review_df

The `style` column can now be dropped since it has been dummy encoded.

In [ ]:
review_df.drop(columns='style', inplace = True)
review_df

### Preprocess `verified` column

In [ ]:
review_df['verified'].value_counts()

Since the verified column contains only true and false, the datatype can be changed to int8 for the regression analysis

In [ ]:
# Map Trues to 1 and false to 0

review_df['verified'] = review_df['verified'].astype('int8')

In [ ]:
review_df.info()

### Narrowing down Dataset

To narrow in the recommendation capacities, only the movies with > 1000 reviews will be looked at for now.

In [ ]:
review_df['asin'].value_counts()

In [ ]:
# Determine the review occurance for every item 
# Output the index and the count for # of reviews for every row.
review_df.groupby('asin')['asin'].transform('size')

From the above, it can be seen that there are 15,434 unique items from the 2 million rows of reviews.

In [ ]:
item_subset = review_df.groupby('asin')['asin'].transform('size') >= 100

# Check results

item_subset

In [ ]:
new_review_df = review_df[item_subset].copy()

# Check results
new_review_df

Reset the index

In [ ]:
new_review_df.reset_index(drop = True, inplace=True)

Check that all items have over 1000 reviews

In [ ]:
new_review_df['asin'].value_counts()

There are 3744 unique items and all of them have atleast 100 reviews

### Create a clean df that only has numeric values for exploratory data analysis

In [ ]:
# Seperate out the numeric columns
numeric_col = list(new_review_df.select_dtypes("number").columns)

# Make a new clean dataframe with only the numeric columns
clean_df = new_review_df[numeric_col].copy()

# Check results
clean_df

### Preprocess reviewerName and ASIN column

Convert the reviewerName column to a numeric representation using `pd.factorize`

In [ ]:
clean_df['reviewer_ID'] = pd.factorize(new_review_df['reviewerID'])[0]

Do the same for the product column (ASIN)

In [ ]:
clean_df['itemID'] = pd.factorize(new_review_df['asin'])[0]

In [ ]:
clean_df.head()

The preprocessing for the review data is now complete and the EDA and regression analysis can now be performed.

## Save the data to a JSON file

In [ ]:
# Save the preprocessed data as one file
new_review_df.to_json(r'preprocessed_review.json')

In [ ]:
# Save the numeric data as another file
clean_df.to_json(r'numeric_review.json')

# 3. Exploratory Data Analysis <a class ='anchor' id='3EDA'></a>

The distribution for each column can be analyzed by plotting the histograms for each column

In [ ]:
for column in clean_df.columns:
    sns.histplot(x = column, data = clean_df)
    plt.title(column)
    plt.show()

Non of the data in the columns look normally distributed. Thus the linear regression model will fit the data poorly. The `reviewScore` is left skewed with most of the data rated at 5. The `unixReviewTime` is also left skewed with a spike in reviews in the unixTime of about 1.35e9. The `vote` column seems to only have one bar close to 0 which could indicate that the majority of the reviews had low votes from other Amazon users and there is a very small percentage of high vote counts. This datetime represents 2012 which could be due to a spike in Amazon usage. The `style` data only contains 1s and 0s since they were dummy encoded thus, it doesn't make sense to look at the histograms for those data. (Histograms are used for continuous data). The `reviewer_ID` and `ItemID` columns are not ordinal data, so looking at a histogram can be misleading. Spikes in the data just mean a person is reviewing a lot of products for `reviewer_ID` and spikes in `itemID` mean that some products are being reviewed a lot of times. 

In [ ]:
datetime.fromtimestamp(1350000000)

### Check the values in the vote column

In [ ]:
clean_df['vote'].value_counts()

Most reviews had 0 votes and many reviews had below 5 votes.

In [ ]:
clean_df['vote'].value_counts().head(1)/clean_df.shape[0]

About 79% of the reviews have 0 votes which is an extremely high percentage that will certainly skew the `vote` data

Show correlations between different features

In [ ]:
plt.figure(figsize= [12,8])
sns.heatmap(clean_df.corr().round(2), vmin=-1, vmax=1, cmap='coolwarm', annot=True)
plt.show()

## Conclusion for Preprocessing and EDA

In [ ]:
count vectorize meta_data and count vectorize review data separately. Add them together afterwards.

## Read in the clean meta data and combine it with the clean review data

In [ ]:
# meta_df = pd.read_csv('clean_meta.csv')
# meta_df

In [ ]:
# clean_df['asin'] = review_df['asin']

In [ ]:
# new_df = pd.merge(clean_df, meta_df,  how='left', left_on='asin', right_on = 'asin')